In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
pd.options.mode.chained_assignment = None  # default='warn'?

# data_key DataFrame
data_key = pd.read_csv('key.csv')

# data_train DataFrame
data_train = pd.read_csv('train.csv')

# data_weather DataFrame
data_weather = pd.read_csv('weather.csv')

rain_text = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'FG+']
other_text = ['HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC' ]

data_weather['codesum'].replace("+", "")
a = []
for i in range(len(data_weather['codesum'])):
    a.append(data_weather['codesum'].values[i].split(" "))
    for i_text in a[i]:
        if len(i_text) == 4:
            a[i].append(i_text[:2])
            a[i].append(i_text[2:])
            
data_weather["nothing"] = 1
data_weather["rain"] = 0
data_weather["other"] = 0
b = -1
for ls in a:
    b += 1
    for text in ls:
        if text in rain_text:
            data_weather.loc[b, 'rain'] = 1
            data_weather.loc[b, 'nothing'] = 0
        elif text in other_text:
            data_weather.loc[b,'other'] = 1
            data_weather.loc[b, 'nothing'] = 0      
# 모든 데이터 Merge
df = pd.merge(data_weather, data_key)

station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr

df = pd.merge(df, data_train)

# T 값 처리 하기. Remained Subject = > 'M' and '-'
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005

# 주말과 주중 구분 작업 하기
df['date'] = pd.to_datetime(df['date'])
df['week7'] = df['date'].dt.dayofweek
df['weekend'] = 0
df.loc[df['week7'] == 5, 'weekend'] = 1
df.loc[df['week7'] == 6, 'weekend'] = 1

df1 = df[df['station_nbr'] == 1]; df11 = df[df['station_nbr'] == 11]
df2 = df[df['station_nbr'] == 2]; df12 = df[df['station_nbr'] == 12]
df3 = df[df['station_nbr'] == 3]; df13 = df[df['station_nbr'] == 13]
df4 = df[df['station_nbr'] == 4]; df14 = df[df['station_nbr'] == 14]
df5 = df[df['station_nbr'] == 5]; df15 = df[df['station_nbr'] == 15]
df6 = df[df['station_nbr'] == 6]; df16 = df[df['station_nbr'] == 16]
df7 = df[df['station_nbr'] == 7]; df17 = df[df['station_nbr'] == 17]
df8 = df[df['station_nbr'] == 8]; df18 = df[df['station_nbr'] == 18]
df9 = df[df['station_nbr'] == 9]; df19 = df[df['station_nbr'] == 19]
df10 = df[df['station_nbr'] == 10]; df20 = df[df['station_nbr'] == 20]

In [4]:
df11 = df11.apply(pd.to_numeric, errors = 'coerce')
df11.describe().iloc[:, 14:]
# 없는 Column = date, codesum, snowfall, station_nbr

,stnpressure,sealevel,resultspeed,resultdir,avgspeed,nothing,rain,other,store_nbr,station_nbr,item_nbr,units,week7,weekend
count,201132.000000,189144.000000,189144.000000,189144.000000,201576.000000,202242.000000,202242.000000,202242.000000,202242.000000,202242.0,202242.000000,202242.000000,202242.000000,202242.000000
mean,30.017583,30.063815,5.090610,17.269953,6.243943,0.431394,0.565313,0.036224,27.500000,11.0,56.000000,0.518369,3.001098,0.286498
std,0.129627,0.129079,2.924593,9.958385,2.745724,0.495272,0.495717,0.186847,15.500038,0.0,32.041719,4.476756,2.003021,0.452126
min,29.520000,29.570000,0.200000,1.000000,1.200000,0.000000,0.000000,0.000000,12.000000,11.0,1.000000,0.000000,0.000000,0.000000
25%,29.930000,29.980000,2.800000,8.000000,4.300000,0.000000,0.000000,0.000000,12.000000,11.0,28.000000,0.000000,1.000000,0.000000
50%,30.010000,30.060000,4.600000,17.000000,5.700000,0.000000,1.000000,0.000000,27.500000,11.0,56.000000,0.000000,3.000000,0.000000
75%,30.100000,30.140000,6.900000,25.000000,7.700000,1.000000,1.000000,0.000000,43.000000,11.0,84.000000,0.000000,5.000000,1.000000
max,30.420000,30.470000,16.200000,36.000000,17.400000,1.000000,1.000000,1.000000,43.000000,11.0,111.000000,132.000000,6.000000,1.000000


In [3]:
df11['store_nbr'].unique()

array([12, 43], dtype=int64)

In [5]:
df11_drop_columns = ['codesum', 'station_nbr', 'date']
df11 = df11.drop(columns = df11_drop_columns)

In [6]:
# np.nan를 포함하고 있는 변수(column)를 찾아서, 그 변수에 mean 값 대입해서 Frame의 모든 Value가 fill 되게 하기.
df11_columns = df11.columns

# Cateogry 값을 포함하는 변수는 np.nan에 mode값으로 대체하고, 나머지 실수 값을 포함한 변수는 np.nan에 mean값으로 대체
for i in df11_columns:
    if (i == 'resultdir'):
        df11[i].fillna(df11[i].mode()[0], inplace=True)
        print(df11[i].mode()[0])
    else:
        df11[i].fillna(df11[i].mean(), inplace=True)

# 이제 모든 변수가 숫자로 표기 되었기 때문에, 가능 함. 
# 상대 습도 추가 #
df11['relative_humility'] = 100*(np.exp((17.625*((df11['dewpoint']-32)/1.8))/(243.04+((df11['dewpoint']-32)/1.8)))/np.exp((17.625*((df11['tavg']-32)/1.8))/(243.04+((df11['tavg']-32)/1.8))))

# 체감온도 계산
df11["windchill"] = 35.74 + 0.6215*df11["tavg"] - 35.75*(df11["avgspeed"]**0.16) + 0.4275*df11["tavg"]*(df11["avgspeed"]**0.16)

df11 = df11[df11['units'] != 0]

4.0


In [7]:
# 'np.log1p(units) ~ tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + sunset + codesume + \
# snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir + avgspeed + nothing + rain + other + \
# store_nbr + station_nbr + item_nbr'

model_df11 = sm.OLS.from_formula('np.log1p(units) ~ depart + tmax + tmin + tavg + dewpoint + heat + cool + preciptotal + \
resultspeed + sealevel + snowfall + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data = df11)
result_df11 = model_df11.fit()

print(result_df11.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.823
Model:                            OLS   Adj. R-squared:                  0.822
Method:                 Least Squares   F-statistic:                     757.9
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        15:52:23   Log-Likelihood:                -3044.9
No. Observations:                4924   AIC:                             6152.
Df Residuals:                    4893   BIC:                             6353.
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(nothing)[0]          0.0393      0

In [8]:
model_df11 = sm.OLS.from_formula('np.log1p(units) ~ scale(depart) + scale(snowfall) + scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(heat) + scale(cool) + \
scale(preciptotal) + scale(resultspeed) + scale(sealevel) + scale(resultdir) + scale(avgspeed) + C(nothing) + C(rain) + \
C(other) + C(item_nbr) + C(week7) + C(weekend) + scale(relative_humility) + scale(windchill) + 0', data = df11)
result_df11 = model_df11.fit()

print(result_df11.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.823
Model:                            OLS   Adj. R-squared:                  0.822
Method:                 Least Squares   F-statistic:                     757.9
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        15:52:55   Log-Likelihood:                -3044.9
No. Observations:                4924   AIC:                             6152.
Df Residuals:                    4893   BIC:                             6353.
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(nothing)[0]           

In [9]:
anova_result_df11 = sm.stats.anova_lm(result_df11).sort_values(by=['PR(>F)'], ascending = False)
anova_result_df11[anova_result_df11['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
scale(avgspeed),1.0,0.899609,0.899609,4.432767,3.530614e-02
scale(sealevel),1.0,0.942568,0.942568,4.644440,3.120293e-02
C(rain),1.0,1.061884,1.061884,5.232364,2.221287e-02
scale(windchill),1.0,1.125688,1.125688,5.546756,1.855424e-02
scale(tmin),1.0,1.204094,1.204094,5.933095,1.489458e-02
scale(cool),1.0,1.553990,1.553990,7.657185,5.675916e-03
scale(heat),1.0,3.791245,3.791245,18.681112,1.575724e-05
scale(dewpoint),1.0,4.550746,4.550746,22.423503,2.248599e-06
scale(depart),1.0,4.830217,4.830217,23.800579,1.102407e-06
C(other),1.0,5.546938,5.546938,27.332174,1.784671e-07


In [10]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df11.values, i) for i in range(df11.shape[1])]
vif["features"] = df11.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif
# 10순위까지 겹치는것만 쓴다
# C(other), resultdir, preciptotal, C(week7), C(item_nbr), depart

,VIF Factor,features
0,1.026938,store_nbr
1,1.068242,snowfall
2,1.252931,preciptotal
3,1.267900,other
4,1.369660,resultdir
5,1.558447,item_nbr
6,1.597895,units
7,2.724129,depart
8,2.746743,week7
9,2.788873,weekend


In [12]:
# resultdir, preciptotal, C(rain), C(week7), C(item_nbr)
model_df11 = sm.OLS.from_formula('np.log1p(units) ~ C(item_nbr) + C(other) + C(week7)+ scale(resultdir) + scale(preciptotal) + \
+ scale(depart) + 0', data = df11)
result_df11 = model_df11.fit()

print(result_df11.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.819
Model:                            OLS   Adj. R-squared:                  0.818
Method:                 Least Squares   F-statistic:                     1306.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        15:56:10   Log-Likelihood:                -3098.9
No. Observations:                4924   AIC:                             6234.
Df Residuals:                    4906   BIC:                             6351.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(item_nbr)[3]         1.1565      0

In [14]:
# resultdir, preciptotal, C(rain), C(week7), C(item_nbr)
X11 = df11[['item_nbr', 'other', 'week7', 'resultdir', 'preciptotal', 'depart']]
y11 = df11['units']
model11 = LinearRegression()

cv11 = KFold(n_splits=11, shuffle=True, random_state=0)

cross_val_score(model11, X11, y11, scoring="r2", cv=cv11)

array([0.29679055, 0.35691551, 0.34805084, 0.28408039, 0.34981776,
       0.24258367, 0.30773128, 0.36661761, 0.31596591, 0.30153076,
       0.35926033])